# Tracking The Space Station In Real Time

<span class="pubdate">March 2016</span>

Do you know where the [International Space Station](http://www.nasa.gov/audience/forstudents/5-8/features/nasa-knows/what-is-the-iss-58.html) is right now? There are lots of sites[^1] that claim to show the current position of the ISS. NASA probably knows where it is[^2], but how is it done?

[^1]: <https://www.google.com/webhp?q=The+current+position+of+the+ISS>
[^2]: Citation Needed

## Early History Of Satellite Tracking

In the 1950's scientists and engineers (and science fiction writters) realsized that building a rocket that could launch an experiment in to Earth orbit would be practical in the near future.

In [ ]:
# dependencies
from numpy import loadtxt, column_stack, subtract, multiply, divide, average
from numpy.linalg import norm
from numpy import mgrid, pi, sin, cos
import datetime
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from mpl_toolkits.mplot3d import Axes3D
%matplotlib inline

# constants
GPS2UNIX_OFFSET  = 315964800  # sec
RADIUS_EARTH     =      6371  # km

date_fmt = mdates.DateFormatter('%b-%d %H:%M')
red = "#941e56"
blue = "#4a73e1"
green = "#22b464"

u, v = mgrid[0:2*pi:40j, 0:pi:20j]
radius = RADIUS_EARTH
es_x = cos(u)*sin(v)*radius
es_y = sin(u)*sin(v)*radius
es_z = cos(v)*radius

In [ ]:
# import data
columns = loadtxt('./data/ISS-Live-Capture.csv', delimiter=',', unpack=True)

receive_time = columns[0]
state_time   = columns[1]
state_x, state_y, state_z = columns[2:5]

# converty GPS stamp to datetime object
state_time = [datetime.datetime.utcfromtimestamp(t + GPS2UNIX_OFFSET) for t in state_time]

# vector position
state_vec = column_stack(( state_x,  state_y,  state_z))
radius = norm(state_vec, axis=1)

# find gaps
state_gaps = []
for i, t in enumerate(state_time[:-1]):
    diff = (state_time[i+1] - t).seconds
    if diff > 15:
        state_gaps.append(i)

Gathering ISS state vector data from [isslive.com](http://isslive.com/displays/adcoDisplay1.html).

## The Data

Downloads:

 - This article, source code, and all data: [archive.zip](https://github.com/natronics/ISS-TLE-Propagation/archive/gh-pages.zip), [archive.tar.gz](https://github.com/natronics/ISS-TLE-Propagation/archive/gh-pages.tar.gz)
 - [GitHub project page](https://github.com/natronics/ISS-TLE-Propagation)

See the position vector over time

In [ ]:
fig, ax1 = plt.subplots(figsize=(24,8))
plt.title(r"ISS State Vector J2000 X,Y,Z Position Over Time")
plt.ylabel(r"J2000 Coordinate [km]")
plt.xlabel(r"State Vector Date [Month-Day HH:MM (UTC)]")

begin_data = 0
for end_data in state_gaps:
    plt.plot(state_time[begin_data:end_data], state_x[begin_data:end_data], color=red, alpha=0.75)
    plt.plot(state_time[begin_data:end_data], state_y[begin_data:end_data], color=blue, alpha=0.75)
    plt.plot(state_time[begin_data:end_data], state_z[begin_data:end_data], color=green, alpha=0.75)
    begin_data = end_data+1

# hack for labels
plt.plot([state_time[0],state_time[0]], [-100000,-110000], color=red, alpha=0.8, label="X")
plt.plot([state_time[0],state_time[0]], [-100000,-110000], color=blue, alpha=0.8, label="Y")
plt.plot([state_time[0],state_time[0]], [-100000,-110000], color=green, alpha=0.8, label="Z")

ax1.xaxis.set_major_formatter(date_fmt)
plt.setp(ax1.get_xticklabels(), rotation=45, horizontalalignment='right')
plt.ylim([-10e3,10e3])
ax1.legend(loc=1, title="Coordinate:")
plt.show()

In [ ]:
fig, ax1 = plt.subplots(figsize=(24,24))
ax1 = plt.axes(projection='3d')
plt.title(r"ISS Orbit")
plt.xlabel(r"J2000 X [km]")
plt.ylabel(r"J2000 Y [km]")
ax1.set_zlabel('J2000 Z [km]')

ax1.plot_wireframe(es_x, es_y, es_z, color=blue, alpha=0.2, lw=1.2)

begin_data = 0
for end_data in state_gaps:
    ax1.plot(state_x[begin_data:end_data], state_y[begin_data:end_data], state_z[begin_data:end_data], '-', color=red, alpha=0.7, lw=0.4)
    begin_data = end_data+1

ax1.view_init(elev=15.0, azim=60)
plt.setp(ax1.get_zticklabels(), horizontalalignment='right')
ax1.set_xlim(-6000, 6000)
ax1.set_ylim(-6000, 6000)
ax1.set_zlim(-6000, 6000)
ax1.set_aspect('equal','box')
plt.show()

See the alititude over time

In [ ]:
height = subtract(radius, RADIUS_EARTH)

avg_height = average(height)
fig, ax1 = plt.subplots(figsize=(24,8))
plt.title(r"ISS Geocentric Altitude Over Time")
plt.ylabel(r"Geocentric Altitude [km]")
plt.xlabel(r"State Vector Date [Month-Day HH:MM (UTC)]")

begin_data = 0
for end_data in state_gaps:
    plt.plot(state_time[begin_data:end_data], height[begin_data:end_data], alpha=0.8, color=red)
    begin_data = end_data+1

# label hack
plt.plot([state_time[0],state_time[0]],[0,0], color=red, alpha=0.8, label="Geocentric Altitude")

plt.plot([state_time[0], state_time[-1]], [avg_height, avg_height], 'k--', alpha=0.4, label="Mean Geocentric Altitude (%0.1f) km" % avg_height)

ax1.xaxis.set_major_formatter(date_fmt)
plt.setp(ax1.get_xticklabels(), rotation=45, horizontalalignment='right')
plt.ylim([400,425])
ax1.legend(loc=1)
plt.show()

In [ ]:
velocity_vec = []
velocity_pos = []
velocity = []
_v = []
for i, t in enumerate(state_time[:-1]):
    delta_t = (state_time[i+1] - t)
    delta_t = delta_t.seconds + (delta_t.microseconds / 1e6)
    if delta_t < 15:
        s = state_vec[i] - state_vec[i+1]
        v = multiply(divide(s, delta_t), 1000.0)
        velocity_vec.append(v)
        velocity_pos.append(s)
        velocity.append(norm(v))
        _v.append(norm(v))
    else:
        velocity.append(None)
velocity.append(None)
avg_velocity = average(_v)

In [ ]:
fig, ax1 = plt.subplots(figsize=(24,8))
plt.title(r"ISS Velocity Over Time")
plt.ylabel(r"ECI Velocity [m/s]")
plt.xlabel(r"State Vector Date [Month-Day HH:MM (UTC)]")


ax1.plot(state_time, velocity, color=red, alpha=0.8, label="ECI Velocity")
ax1.plot([state_time[0], state_time[-1]], [avg_velocity, avg_velocity], 'k--', alpha=0.4, label="Mean ECI Velocity (%0.1f) m/s" % avg_velocity)

# label hack
ax1.plot([state_time[0],state_time[0]], [0,0], color=blue, linestyle='-.', alpha=0.3, label="Geocentric Altitude")

ax1.xaxis.set_major_formatter(date_fmt)
plt.setp(ax1.get_xticklabels(), rotation=45, horizontalalignment='right')
plt.ylim([7650,7685])
ax1.legend(loc=1)

ax2 = ax1.twinx()
begin_data = 0
for end_data in state_gaps:
    ax2.plot(state_time[begin_data:end_data], height[begin_data:end_data], color=blue, linestyle='-.', alpha=0.3)
    begin_data = end_data+1
ax2.grid(b=False)
plt.ylim([400,425])
plt.ylabel(r"Geocentric Altitude [km]")

plt.show()

--------------------------------------------------------------------------------------

This article was written in
[Jupyter Notebooks](http://jupyter.org/).
[View the original document](http://nbviewer.jupyter.org/github/natronics/ISS-TLE-Propagation/blob/gh-pages/index.ipynb) or download the raw file: [index.ipynb](https://raw.githubusercontent.com/natronics/ISS-TLE-Propagation/gh-pages/index.ipynb)

Did I make a mistake? Have something to add? You can
[create an issue](https://github.com/natronics/ISS-TLE-Propagation/issues)
, or
[build this page yourself](https://github.com/natronics/ISS-TLE-Propagation/blob/gh-pages/README.markdown)
and
[send a pull request](https://help.github.com/categories/collaborating-on-projects-using-pull-requests/).

<a rel="license" href="https://creativecommons.org/licenses/by-nc/4.0/"><img class="wrap" alt="Creative Commons License" style="border-width:0" src="https://i.creativecommons.org/l/by-nc/4.0/80x15.png" /></a> &nbsp; This <span xmlns:dct="https://purl.org/dc/terms/" href="https://purl.org/dc/dcmitype/InteractiveResource" rel="dct:type">work</span> by <a xmlns:cc="https://creativecommons.org/ns#" href="https://github.com/natronics" property="cc:attributionName" rel="cc:attributionURL">natronics</a> is licensed under a <a rel="license" href="https://creativecommons.org/licenses/by-nc/4.0/">Creative Commons Attribution-NonCommercial 4.0 International License</a>.

#### Footnotes